In [ ]:
!pip install transformers
!pip install bertopic
!pip install sentence_transformers

ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
from bertopic import BERTopic
from collections import Counter

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined_transcipts.csv to combined_transcipts.csv


In [ ]:
data = pd.read_csv('combined_transcipts.csv')

In [ ]:
data.head()

,Video_URL,Youtuber,Post_date,Full_Transcript
0,https://www.youtube.com/watch?v=-Kwbd-Duzmg,Kamala Harris,"Oct 17, 2024",when I was five I began getting sexually abuse...
1,https://www.youtube.com/watch?v=-XINcrcvrMU,Kamala Harris,"Nov 1, 2024",you me if you want to these expensive is red b...
2,https://www.youtube.com/watch?v=-j7X6mjrJ3M,Kamala Harris,"Oct 11, 2024",so my question for you is what are your plans ...
3,https://www.youtube.com/watch?v=-vEE44TNMqE,Kamala Harris,"Oct 6, 2024",we fight like hell and if you don't fight like...
4,https://www.youtube.com/watch?v=0RLhu9Ngchc,Donald J Trump,"Jul 21, 2024",kamla was in on it she covered up Joe's obviou...


Pre trained model for Topic and Tone Analysis

In [ ]:
!pip install bertopic

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", device=0 if device == "cuda" else -1)
emotion_analyzer = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", device=0 if device == "cuda" else -1)
hate_speech_analyzer = pipeline("text-classification", model="Hate-speech-CNERG/bert-base-uncased-hatexplain", device=0 if device == "cuda" else -1)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
topic_model = BERTopic()

Chunking because the transcripts are way longer than 512 tokens

In [ ]:
def chunk_text(text, tokenizer, max_length=510):
    tokens = tokenizer(text, add_special_tokens=False)['input_ids']
    chunks = [tokens[i:i + max_length - 2] for i in range(0, len(tokens), max_length - 2)]
    chunks_with_special_tokens = [
        tokenizer.decode([tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id], skip_special_tokens=False)
        for chunk in chunks
    ]
    return chunks_with_special_tokens

In [ ]:
def aggregate_results(results):
    counter = Counter(results)
    return counter.most_common(1)[0][0]

Prepare single transcripts

In [ ]:
transcripts = data['Full_Transcript'].tolist()

toipc analysis

In [ ]:
topics, probs = topic_model.fit_transform(transcripts)  # Analyze topics
data['Topic'] = topics


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

analyze tone, emotion, hate speech

In [ ]:
sentiments, emotions, hate_speech_labels = [], [], []

In [ ]:
for transcript in transcripts:
    # Split the transcript into valid chunks
    chunks = chunk_text(transcript, tokenizer)

    # Sentiment Analysis
    chunk_sentiments = [sentiment_analyzer(chunk)[0]['label'] for chunk in chunks]
    aggregated_sentiment = aggregate_results(chunk_sentiments)
    sentiments.append(aggregated_sentiment)

    # Emotion Analysis
    chunk_emotions = [emotion_analyzer(chunk)[0]['label'] for chunk in chunks]
    aggregated_emotion = aggregate_results(chunk_emotions)
    emotions.append(aggregated_emotion)

    # Hate Speech Detection
    chunk_hate_speech = [hate_speech_analyzer(chunk)[0]['label'] for chunk in chunks]
    aggregated_hate_speech = aggregate_results(chunk_hate_speech)
    hate_speech_labels.append(aggregated_hate_speech)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Token indices sequence length is longer than the specified maximum sequence length for this model (1543 > 512). Running this sequence through the model will result in indexing errors
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Add results to data frame




In [ ]:
data['Sentiment'] = sentiments
data['Emotion'] = emotions
data['Hate_Speech_Label'] = hate_speech_labels

In [ ]:
data.head()

,Video_URL,Youtuber,Post_date,Full_Transcript,Topic,Sentiment,Emotion,Hate_Speech_Label
0,https://www.youtube.com/watch?v=-Kwbd-Duzmg,Kamala Harris,"Oct 17, 2024",when I was five I began getting sexually abuse...,2,POSITIVE,sadness,normal
1,https://www.youtube.com/watch?v=-XINcrcvrMU,Kamala Harris,"Nov 1, 2024",you me if you want to these expensive is red b...,0,NEGATIVE,joy,normal
2,https://www.youtube.com/watch?v=-j7X6mjrJ3M,Kamala Harris,"Oct 11, 2024",so my question for you is what are your plans ...,2,NEGATIVE,sadness,normal
3,https://www.youtube.com/watch?v=-vEE44TNMqE,Kamala Harris,"Oct 6, 2024",we fight like hell and if you don't fight like...,0,NEGATIVE,anger,normal
4,https://www.youtube.com/watch?v=0RLhu9Ngchc,Donald J Trump,"Jul 21, 2024",kamla was in on it she covered up Joe's obviou...,1,NEGATIVE,sadness,normal


In [ ]:
topic_info = topic_model.get_topic_info()
print(topic_info)

   Topic  Count              Name  \
0     -1     31  -1_the_and_to_of   
1      0     84  0_the_and_to_you   
2      1     83  1_the_and_to_you   
3      2     23   2_to_the_and_of   

                                    Representation  \
0  [the, and, to, of, you, that, in, we, it, have]   
1    [the, and, to, you, that, of, in, we, it, is]   
2  [the, and, to, you, that, of, it, they, in, we]   
3   [to, the, and, of, in, my, that, her, we, for]   

                                 Representative_Docs  
0  [But that's beside the point. I mean you got a...  
1  [Madam vice president thank you for the time t...  
2  [Mr President welcome back to the Cudo show we...  
3  [so so let me take a minute to help folks espe...  


as topic modelling still lacks, I continue as following

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk


In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
    """
    Preprocess text by lowercasing, removing stopwords, and lemmatizing.
    """
    # Initialize lemmatizer and stopwords
    lemmatizer = WordNetLemmatizer()
    custom_stopwords = set(stopwords.words('english')).union(ENGLISH_STOP_WORDS)
    special_stopwords = {'donald', 'trump', 'harris', 'kamala', 'people', 'president', 'kamla', 'america', 'great', 'american', 'year', 'country'}
    custom_stopwords = custom_stopwords.union(special_stopwords)

    # Lowercase
    text = text.lower()

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stopwords]

    return ' '.join(tokens)

In [ ]:
data['Processed_Transcript'] = data['Full_Transcript'].apply(preprocess_text)

Adjust Bertopic parameters

In [ ]:
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight, effective embedding model



In [ ]:
topic_model = BERTopic(
    embedding_model=embedding_model,
    n_gram_range=(1,1),
    min_topic_size=5,
    top_n_words=10,
    calculate_probabilities=True )

In [ ]:
topics, probs = topic_model.fit_transform(data['Processed_Transcript'].tolist())


In [ ]:
data['Topic'] = topics

Results

In [ ]:
topic_info = topic_model.get_topic_info()
print(topic_info)


   Topic  Count                                     Name  \
0     -1     20              -1_going_know_country_thats   
1      0     99                     0_know_uh_going_like   
2      1     41  1_kamla_message_approve message_approve   
3      2     28                     2_going_know_said_uh   
4      3     17               3_woman_abortion_state_ban   
5      4     16                   4_care_know_right_like   

                                      Representation  \
0  [going, know, country, thats, want, theyre, sa...   
1  [know, uh, going, like, think, country, dont, ...   
2  [kamla, message, approve message, approve, im,...   
3  [going, know, said, uh, country, tariff, dont,...   
4  [woman, abortion, state, ban, life, pregnancy,...   
5  [care, know, right, like, child, home, plan, m...   

                                 Representative_Docs  
0  [think moment hardship beauty country really r...  
1  [right right like apologize tonight audience v...  
2  [music thing kamla

already better - maybe improves with only nouns

In [ ]:
!pip install spacy
!-m spacy download en_core_web_sm

/bin/bash: - : invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--pretty-print
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


In [ ]:
import spacy

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

def extract_key_words(text):
    """
    Extract meaningful nouns, proper nouns, and key adjectives from the text.
    """
    doc = nlp(text.lower())
    allowed_pos = {'NOUN', 'PROPN', 'ADJ'}  # Keep nouns, proper nouns, and adjectives
    key_words = [token.lemma_ for token in doc if token.pos_ in allowed_pos]
    return ' '.join(key_words)

data['Key_Words_Transcript'] = data['Processed_Transcript'].apply(extract_key_words)

In [ ]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=5, cluster_selection_epsilon=0.01)

In [ ]:
topic_model = BERTopic(hdbscan_model=hdbscan_model,
                       embedding_model=embedding_model,
                       n_gram_range=(1, 2),
                       min_topic_size=10,
                       top_n_words=10)


In [ ]:
topics, probs = topic_model.fit_transform(data['Key_Words_Transcript'].tolist())
data['Topic'] = topics

In [ ]:
topic_model.reduce_topics(data['Key_Words_Transcript'].tolist(), nr_topics=7)

In [ ]:
topic_info = topic_model.get_topic_info()
print(topic_info)

   Topic  Count                                        Name  \
0     -1     23                -1_puerto_nation_biomic_fund   
1      0     55                    0_care_price_plan_middle   
2      1     48                       1_thing_good_lot_time   
3      2     37                  2_freedom_music_vote_state   
4      3     28                 3_woman_abortion_state_life   
5      4     13                  4_border_member_gang_agent   
6      5     11  5_vice_national security_dictator_security   
7      6      6                6_fema_work_hurricane_ground   

                                      Representation  \
0  [puerto, nation, biomic, fund, rico, freedom, ...   
1  [care, price, plan, middle, class, tax, middle...   
2  [thing, good, lot, time, state, way, year, kno...   
3  [freedom, music, vote, state, election, strong...   
4  [woman, abortion, state, life, daughter, famil...   
5  [border, member, gang, agent, immigrant, immig...   
6  [vice, national security, dictator, s

In [ ]:
data.head()

,Video_URL,Youtuber,Post_date,Full_Transcript,Topic,Sentiment,Emotion,Hate_Speech_Label,Processed_Transcript,Nouns_Transcript,Key_Words_Transcript
0,https://www.youtube.com/watch?v=-Kwbd-Duzmg,Kamala Harris,"Oct 17, 2024",when I was five I began getting sexually abuse...,3,POSITIVE,sadness,normal,began getting sexually abused stepfather got p...,stepfather music pregnancy rape state abortion...,stepfather pregnant music pregnancy rape state...
1,https://www.youtube.com/watch?v=-XINcrcvrMU,Kamala Harris,"Nov 1, 2024",you me if you want to these expensive is red b...,1,NEGATIVE,joy,normal,want expensive red bottom blood shoe hit want ...,bottom blood shoe money applause applause guy ...,expensive red bottom blood shoe applause music...
2,https://www.youtube.com/watch?v=-j7X6mjrJ3M,Kamala Harris,"Oct 11, 2024",so my question for you is what are your plans ...,4,NEGATIVE,sadness,normal,question plan plan support subgroup immigrant ...,question plan plan subgroup immigrant life sha...,question plan plan support subgroup immigrant ...
3,https://www.youtube.com/watch?v=-vEE44TNMqE,Kamala Harris,"Oct 6, 2024",we fight like hell and if you don't fight like...,2,NEGATIVE,anger,normal,fight like hell dont fight like hell youre goi...,hell hell country evidence effort election bom...,hell hell new evidence trump effort overturn e...
4,https://www.youtube.com/watch?v=0RLhu9Ngchc,Donald J Trump,"Jul 21, 2024",kamla was in on it she covered up Joe's obviou...,-1,NEGATIVE,sadness,normal,covered joes obvious mental decline good shape...,kamla decline president shape health tireless ...,covered obvious mental decline good shape good...


In [ ]:
topic_info.Name

,Name
0,-1_puerto_nation_biomic_fund
1,0_care_price_plan_middle
2,1_thing_good_lot_time
3,2_freedom_music_vote_state
4,3_woman_abortion_state_life
5,4_border_member_gang_agent
6,5_vice_national security_dictator_security
7,6_fema_work_hurricane_ground


In [ ]:
topic_mapping = {}
for topic_id in topic_info['Topic']:
    if topic_id == -1:  # Skip outliers
        topic_mapping[topic_id] = "Outlier"
    else:
        top_words = [word[0] for word in topic_model.get_topic(topic_id)[:3]]  # Top 3 words
        topic_mapping[topic_id] = "_".join(top_words)

data['Topic_Name'] = data['Topic'].map(topic_mapping)


In [ ]:
data = data.drop(columns=['Processed_Transcript', 'Nouns_Transcript', 'Key_Words_Transcript'])
data.to_csv('updated_transcripts.csv', index=False)


In [ ]:
data.head()

,Video_URL,Youtuber,Post_date,Full_Transcript,Topic,Sentiment,Emotion,Hate_Speech_Label,Topic_Name
0,https://www.youtube.com/watch?v=-Kwbd-Duzmg,Kamala Harris,"Oct 17, 2024",when I was five I began getting sexually abuse...,3,POSITIVE,sadness,normal,woman_abortion_state
1,https://www.youtube.com/watch?v=-XINcrcvrMU,Kamala Harris,"Nov 1, 2024",you me if you want to these expensive is red b...,1,NEGATIVE,joy,normal,thing_good_lot
2,https://www.youtube.com/watch?v=-j7X6mjrJ3M,Kamala Harris,"Oct 11, 2024",so my question for you is what are your plans ...,4,NEGATIVE,sadness,normal,border_member_gang
3,https://www.youtube.com/watch?v=-vEE44TNMqE,Kamala Harris,"Oct 6, 2024",we fight like hell and if you don't fight like...,2,NEGATIVE,anger,normal,freedom_music_vote
4,https://www.youtube.com/watch?v=0RLhu9Ngchc,Donald J Trump,"Jul 21, 2024",kamla was in on it she covered up Joe's obviou...,-1,NEGATIVE,sadness,normal,Outlier


In [ ]:
from google.colab import files
files.download('updated_transcripts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Visualisierung

# 1. Topics and Emotions Heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create a crosstab of Topic_Name and Emotion
topic_emotion_counts = pd.crosstab(data['Topic_Name'], data['Emotion'])

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(topic_emotion_counts, annot=True, fmt="d", cmap="coolwarm")
plt.title("Topics and Emotions Heatmap", fontsize=16)
plt.xlabel("Emotion", fontsize=14)
plt.ylabel("Topic", fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


#2. Hate Speech Distribution by Topic

In [ ]:
# Group by Topic_Name and Hate_Speech_Label
hate_speech_distribution = data.groupby(['Topic_Name', 'Hate_Speech_Label']).size().unstack()

# Plot the distribution
hate_speech_distribution.plot(kind='bar', stacked=True, figsize=(12, 8), colormap="Set3")
plt.title("Hate Speech Distribution by Topic", fontsize=16)
plt.xlabel("Topic", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45)
plt.legend(title="Hate Speech Label")
plt.tight_layout()
plt.show()


# 3. Sentiment Breakdown Across Topics

In [ ]:
# Group by Topic_Name and Sentiment
sentiment_distribution = data.groupby(['Topic_Name', 'Sentiment']).size().unstack()

# Plot the distribution
sentiment_distribution.plot(kind='bar', stacked=True, figsize=(12, 8), colormap="viridis")
plt.title("Sentiment Breakdown Across Topics", fontsize=16)
plt.xlabel("Topic", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45)
plt.legend(title="Sentiment")
plt.tight_layout()
plt.show()


# 4. Emotion Word Cloud for Specific Topics

In [ ]:
from wordcloud import WordCloud

# Generate a word cloud for a specific topic
selected_topic = "economy_growth_investment"  # Replace with the desired topic name
topic_data = data[data['Topic_Name'] == selected_topic]

# Combine all transcripts for the selected topic
text = " ".join(topic_data['Full_Transcript'])

# Create the word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white", colormap="coolwarm").generate(text)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title(f"Word Cloud for Topic: {selected_topic}", fontsize=16)
plt.tight_layout()
plt.show()


# 5. Combined Insights Dashboard

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Topics and Emotions
sns.heatmap(topic_emotion_counts, annot=False, cmap="coolwarm", ax=axes[0, 0])
axes[0, 0].set_title("Topics and Emotions Heatmap")
axes[0, 0].set_xlabel("Emotion")
axes[0, 0].set_ylabel("Topic")

# Plot 2: Hate Speech Distribution
hate_speech_distribution.plot(kind='bar', stacked=True, ax=axes[0, 1], colormap="Set3", legend=False)
axes[0, 1].set_title("Hate Speech Distribution by Topic")
axes[0, 1].set_xlabel("Topic")
axes[0, 1].set_ylabel("Count")

# Plot 3: Sentiment Breakdown
sentiment_distribution.plot(kind='bar', stacked=True, ax=axes[1, 0], colormap="viridis", legend=False)
axes[1, 0].set_title("Sentiment Breakdown Across Topics")
axes[1, 0].set_xlabel("Topic")
axes[1, 0].set_ylabel("Count")

# Plot 4: Word Cloud for Selected Topic
selected_topic = "economy_growth_investment"
topic_text = " ".join(data[data['Topic_Name'] == selected_topic]['Full_Transcript'])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(topic_text)
axes[1, 1].imshow(wordcloud, interpolation="bilinear")
axes[1, 1].axis("off")
axes[1, 1].set_title(f"Word Cloud for Topic: {selected_topic}")

plt.tight_layout()
plt.show()


In [ ]:
# 6. Interactive Plots for Presentations

In [ ]:
import plotly.express as px

# Create an interactive heatmap for topics and emotions
fig = px.imshow(topic_emotion_counts,
                labels=dict(x="Emotion", y="Topic", color="Count"),
                title="Topics and Emotions Heatmap")
fig.show()


# 7. Key Takeaways Poster Chart

In [ ]:
# Pie Chart: Hate Speech Proportion
hate_speech_counts = data['Hate_Speech_Label'].value_counts()
hate_speech_counts.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(8, 8), colormap="tab10")
plt.title("Proportion of Hate Speech in the Dataset")
plt.ylabel("")
plt.show()
